In [2]:
import tools.postprocessing as pp

In [3]:
sigs_path = "sigs.txt"
sig_correct_path = "sig_correct.txt"
secrets_path = "secrets.txt"
logfile = "hammer.log"

In [4]:
import glob
import json
import tools.postprocessing as pp

sig_correct = open(sig_correct_path).readline().strip()
# results_files = glob.glob("results_*.json")
results_files = glob.glob("results_2025-02-15_09-57-50.json")

for results_file in results_files:
    j = json.load(open(results_file))
    for x in j:
        if x['hammer_results'] is not None:
            sigs = set(x['hammer_results']['signatures'])
            print(len(sigs))
            for sig in sigs:
                if sig != sig_correct:
                    print(x['date'])
                    is_exploitable = pp.is_exploitable(sig_correct, sig, secrets_path)
                    print(f"Exploitable: {is_exploitable}")


1


In [25]:
# process latest experiment
pp.process_experiment(sig_correct_path, sigs_path, secrets_path, logfile)

Correct signature: 4ba84c7f0c2386258d1df3e780a85335e8443f43718b66d41fcf67b452b8fe06118df40e139fc1cccb8db35328a9d885aed2
Correct signature length: 59606
0 faulted signatures


In [27]:
import glob
import pandas as pd
import zipfile


# Get all exploitable_*.zip files
zip_files = glob.glob('results/*.zip')

candidates = []
for zip_filename in zip_files:
    with zipfile.ZipFile(zip_filename, 'r') as zipf:
        with zipf.open('sig_correct.txt') as sig_correct_file:
            sig_correct = sig_correct_file.readline().decode('utf-8').strip()
        with zipf.open('sig_faulty.txt') as sig_faulty_file:
            sig_faulty = sig_faulty_file.readline().decode('utf-8').strip()
    exploitable = pp.is_exploitable(sig_correct, sig_faulty, secrets_path)
    common_prefix_length = pp.prefix(sig_correct, sig_faulty)
    candidates.append([zip_filename, common_prefix_length, len(sig_faulty), len(sig_correct), exploitable])
    
candidates = sorted(candidates, key=lambda x: x[1], reverse=True)

dataset = pd.DataFrame(candidates, columns = ['ID', 'common_prefix_length', 'Faulty len', 'Correct len', 'is_exploitable'])
    
print(dataset)

Empty DataFrame
Columns: [ID, common_prefix_length, Faulty len, Correct len, is_exploitable]
Index: []


In [28]:
# Analyze faulty sig

sig_field_widths = [('H(m)', 32), ('FORS', 10560)]
for i in range(8):
    sig_field_widths.append((f'WOTS{i}', 2400))
sig_field_widths.append(('msg', 0))

i = 0
for cf, cc in zip(sig_faulty, sig_correct):
    if cf != cc:
        print(f"Bit {i} differs: {cf} != {cc}")
        for field, width in sig_field_widths:
            if i < width:
                print(f"Bit {i} is in field {field}")
                break
            print(f"{i} -= {width}")
            i -= width
        else:
            print(f"Bit {i} is in field msg")
            continue
        break
    i += 1/2

NameError: name 'sig_faulty' is not defined